In [2]:
def readGenome(filename):
    """ Read in the genome from the input FASTA file """
    genome = ''
    with open(filename, 'r') as f:
        for line in f:
            # ignore header line with genome information
            if not line[0] == '>':
                genome += line.rstrip()
    return genome

In [3]:
genome = readGenome('chr1.GRCh38.excerpt.fasta')

In [37]:
def editDistance(p, t):
    # Create distance matrix
    D = []
    for i in range(len(p)+1):
        D.append([0]*(len(t)+1))

    # initialize first row to all zeros, and first column to 0, 1, 2, ... as for edit distance
    for i in range(len(t) + 1):
        D[0][i] = 0
    for i in range(len(p) + 1):
        D[i][0] = i

    # fill in the rest of the matrix
    for i in range(1, len(p)+1):
        for j in range(1, len(t)+1):
            dist_hor  = D[i][j-1] + 1
            dist_vert = D[i-1][j] + 1
            if p[i-1] == t[j-1]:
                dist_diag = D[i-1][j-1]
            else:
                dist_diag = D[i-1][j-1] + 1
            D[i][j] = min(dist_hor, dist_vert, dist_diag)

    # return the minimum value in the bottom row, the minimum edit distance
    return min(D[-1])
    

In [45]:
""" Question 1. What is the edit distance of the best match between 
    pattern GCTGATCGATCGTACG and the excerpt of human chromosome 1? """

""" Question 2. What is the edit distance of the best match between 
    pattern GATTTACCAGATTGAG and the excerpt of human chromosome 1? """

p1 = 'GCTGATCGATCGTACG'
p2 = 'GATTTACCAGATTGAG'
t = genome
edit_distance1 = editDistance(p1, t)
edit_distance2 = editDistance(p2, t)
print(edit_distance1)
print(edit_distance2)

3
2


In [46]:
def overlap(a, b, min_length=3):
    """ Return length of longest suffix of 'a' matching
        a prefix of 'b' that is at least 'min_length'
        characters long.  If no such overlap exists,
        return 0. """
    start = 0  # start all the way at the left
    while True:
        start = a.find(b[:min_length], start)  # look for b's prefix in a
        if start == -1:  # no more occurrences to right
            return 0
        # found occurrence; check for full suffix/prefix match
        if b.startswith(a[start:]):
            return len(a)-start
        start += 1  # move just past previous match

In [50]:
def readFastq(filename):
    sequences = []
    qualities = []
    with open(filename) as fh:
        while True:
            fh.readline()  # skip name line
            seq = fh.readline().rstrip()  # read base sequence
            fh.readline()  # skip placeholder line
            qual = fh.readline().rstrip() # base quality line
            if len(seq) == 0:
                break
            sequences.append(seq)
            qualities.append(qual)
    return sequences, qualities

In [61]:
class Index(object):
    def __init__(self, t, k):
        ''' Create index from all substrings of size 'length' '''
        self.k = k  # k-mer length (k)
        self.index = []
        for i in range(len(t) - k + 1):  # for each k-mer
            self.index.append((t[i:i+k], i))  # add (k-mer, offset) pair
        self.index.sort()  # alphabetize by k-mer
    
    def query(self, p):
        ''' Return index hits for first k-mer of P '''
        kmer = p[:self.k]  # query with first k-mer
        i = bisect.bisect_left(self.index, (kmer, -1))  # binary search
        hits = []
        while i < len(self.index):  # collect matching index entries
            if self.index[i][0] != kmer:
                break
            hits.append(self.index[i][1])
            i += 1
        return hits

In [88]:
sequences, _ = readFastq('ERR266411_1.for_asm.fastq')

In [146]:
""" Question 3. Download and parse the read sequences from the provided Phi-X FASTQ file. 
    Next, find all pairs of reads with an exact suffix/prefix match of length at least 30. 
    Don't overlap a read with itself; if a read has a suffix/prefix match to itself, ignore 
    that match.  Ignore reverse complements. Picture the overlap graph corresponding to the 
    overlaps just calculated.  How many edges are in the graph?  In other words, how many 
    distinct pairs of reads overlap? """

def get_kmers(read, k):
    kmer_set = set()
    for i in range(0, len(read)-k+1):
        kmer_set.add(read[i : i+k])
    return kmer_set


def overlap_all_pairs(reads, k, dict={}):
    for read in reads:
        kmers = get_kmers(read, k)
        for kmer in kmers:
            if kmer not in dict.keys():
                dict[kmer] = set()
            dict[kmer].add(read)
    pairs = []
    for head in reads:
        suffix = head[-k:]
        candidates = dict[suffix]
        for tail in candidates:
            if (not head == tail and overlap(head, tail, k)):
                pairs.append((head, tail))

    return pairs

In [147]:
%%time
pairs = overlap_all_pairs(sequences, 30)
len(pairs)

CPU times: user 699 ms, sys: 12.2 ms, total: 711 ms
Wall time: 710 ms


904746

In [148]:
""" Question 4. Picture the overlap graph corresponding to the overlaps computed for the previous question. 
    How many nodes in this graph have at least one outgoing edge? """

len(set(pair[0] for pair in pairs))

7161